Stock and Option Analysis from my Own Trades

By: Alex Siu, Computer Engineering Student at Purdue University

Hello! In this Jupyter Notebook, I will be analyzing my stock and options trades in the stock market using the Robin-Stocks API and also the Pandas library.

I am an avid trader in the stock market as I like to find interesting companies and what type of innovation and technology they produce to increase they value as a company. I also find that it is the best way to grow your wealth, especially as a younger person. I first started to trading during the COVID pandemic, as it was one of the largest accelerations of growth of the stock market is history.

Data Collection

I first had to login in Robinhood using the Robin-Stocks API.

In [3]:
import robin_stocks.robinhood as r 
import pyotp 
import os 
import pandas as pd
from dotenv import load_dotenv

pd.options.mode.chained_assignment = None

#I use the dotenv library to keep my information in a .env file and hidden
load_dotenv()
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')
TWO_FACTOR_AUTH = os.getenv('TWO_FACTOR_AUTH')

#the pyotp allows for a one time password (like two-factor authentication)
totp = pyotp.TOTP(TWO_FACTOR_AUTH).now()
login = r.login(USERNAME, PASSWORD, mfa_code=totp)

One of the features Robin-Stocks has is to display the current stock holdings I have.

In [4]:
my_stocks = r.build_holdings()
for key,value in my_stocks.items():
    print(key,value)

PLTR {'price': '9.910000', 'quantity': '40.00000000', 'average_buy_price': '18.2992', 'equity': '396.40', 'percent_change': '-45.84', 'intraday_percent_change': '0.00', 'equity_change': '-335.568000', 'type': 'stock', 'name': 'Palantir Technologies', 'id': 'f90de184-4f73-4aad-9a5f-407858013eb1', 'pe_ratio': None, 'percentage': '62.27'}
CHPT {'price': '15.950000', 'quantity': '15.00000000', 'average_buy_price': '18.7483', 'equity': '239.25', 'percent_change': '-14.93', 'intraday_percent_change': '0.00', 'equity_change': '-41.974500', 'type': 'stock', 'name': 'ChargePoint', 'id': '150212ae-15c4-4564-9826-a6b2479f06f4', 'pe_ratio': None, 'percentage': '37.58'}


In addition, I can create CSV files for the stocks and options I have traded in the past. 

In [ ]:
r.export_completed_stock_orders(".") 
r.export_completed_option_orders(".")

With this, we have both of the CSV files needed for analysis. I will be analyzing my stocks trades first. 

In [5]:

df = pd.read_csv(r'C:\Users\14434\Desktop\python\robinhood\excel files\my_stocks.csv')
print(df.head())

  symbol                         date order_type  side  fees  quantity  \
0   SOFI  2022-07-20T13:36:11.787000Z     market  sell  0.00      10.0   
1   SOFI  2022-06-17T18:53:45.320000Z     market  sell  0.00      10.0   
2    HUT  2022-06-17T18:52:42.109000Z     market  sell  0.01      60.0   
3    HUT  2022-05-13T15:20:29.705000Z     market   buy  0.00      20.0   
4    HUT  2022-03-29T18:56:22.996000Z     market   buy  0.00      40.0   

   average_price  
0         6.8020  
1         5.7600  
2         1.6015  
3         2.8280  
4         6.1283  


Within the CSV file, it displays the ticker symbol, the date of purchase/sell, the type of order, the buy or sell of the stock, any fees, the quantity of stock bought, and the average price the stock was bought at. We chose to ignore the fees, as most orders don't contain a fee, and the ones that do are negligible.

We can first find all of the tickers of the stocks I have traded on.

In [6]:
ticker_list = []
for index, row in df.iterrows():
    ticker_list.append(row['symbol'])
ticker_list = sorted(list(set(ticker_list)))
print(ticker_list)

['AMC', 'AMD', 'CCIV', 'CHPT', 'CLNE', 'CLOV', 'CPRX', 'DKNG', 'ENPH', 'FUBO', 'GHVI', 'GME', 'HOOD', 'HUT', 'LPCN', 'MSFT', 'MVST', 'OPK', 'PLTR', 'PTON', 'SNOW', 'SOFI', 'SPI', 'THCB', 'TRCH', 'VKTX', 'WISH']


I have traded on a lot of different types of stocks over the last 2 years. A good portion of them are apart of new and innovative technology, as I like to research on the new types of technology that we can see in the next few years. 

The next step is to create to find the net gain on each stock. I will first use the example of everyone favorite stock, GME.

In [7]:
df['total_cost'] = df['quantity'] * df['average_price']
stock_buy = df.loc[(df['symbol'] == 'GME') & (df['side'] == 'buy')]
sum_buy = stock_buy['total_cost'].sum()
stock_sell = df.loc[(df['symbol'] == 'GME') & (df['side'] == 'sell')]
sum_sell = stock_sell['total_cost'].sum()
print(f'Net Gain from GME: {round((sum_sell - sum_buy), 1)}')

Net Gain from GME: -1182.4


We are able to do this for all of the stocks I have bought or sold.

In [8]:
def total_buy_or_sell(ticker, side):
    stock = df.loc[(df['symbol'] == ticker) & (df['side'] == side)]
    stock['total_cost'] = stock['quantity'] * stock['average_price']
    return stock
def net_gain(ticker):
    stock_buy = total_buy_or_sell(ticker, 'buy')
    sum_buy = stock_buy['total_cost'].sum()
    stock_sell = total_buy_or_sell(ticker, 'sell')
    sum_sell = stock_sell['total_cost'].sum()
    sum_total = round((sum_sell - sum_buy), 1)
    return sum_total

net_gain_ticker = {}
total_gain = 0
for ticker in ticker_list:
    net_val = net_gain(ticker)
    net_gain_ticker.update({ticker : net_val})
    total_gain += net_val

sorted_stock_tuples = sorted(net_gain_ticker.items(), key=lambda item: item[1])
sorted_stock_dict = {k: v for k, v in sorted_stock_tuples}

print(sorted_stock_dict)
print(f'total realized gain/loss: {round(total_gain, 1)}')

{'GME': -1182.4, 'AMC': -610.5, 'PLTR': -501.3, 'GHVI': -403.5, 'CHPT': -300.9, 'HUT': -205.6, 'SPI': -91.2, 'DKNG': -73.9, 'SOFI': -68.8, 'HOOD': -15.6, 'MSFT': -14.9, 'TRCH': -14.0, 'VKTX': -12.6, 'FUBO': -3.0, 'CLNE': -0.5, 'CPRX': -0.2, 'OPK': 4.6, 'AMD': 6.0, 'ENPH': 17.6, 'WISH': 55.0, 'MVST': 73.0, 'LPCN': 104.7, 'PTON': 130.8, 'CLOV': 363.7, 'THCB': 401.0, 'CCIV': 582.8, 'SNOW': 685.9}
total realized gain/loss: -1073.8


The outputs displays the net gain from each stock I have traded on, from the most I have lost on, to the greatest. At the end, I also included my total realized gain on all of the stocks I have traded on. As you can see I'm not the best trader, but it is a good learning experience to have as a younger person.

This is also able to be done for the options I have traded on.

In [9]:
options_df = pd.read_csv(r'C:\Users\14434\Desktop\python\robinhood\excel files\my_options.csv')

print(options_df.head())

  chain_symbol expiration_date  strike_price option_type  side  \
0          SPY      2022-08-03         412.0        call  sell   
1          SPY      2022-08-03         412.0        call   buy   
2          SPY      2022-07-29         400.0         put  sell   
3          SPY      2022-07-29         400.0         put   buy   
4          SPY      2022-07-29         400.0        call  sell   

              order_created_at direction  order_quantity order_type  \
0  2022-08-03T17:44:38.077894Z    credit             1.0      limit   
1  2022-08-02T14:50:17.723609Z     debit             1.0      limit   
2  2022-07-28T15:54:03.983595Z    credit             1.0      limit   
3  2022-07-27T19:46:35.553236Z     debit             1.0      limit   
4  2022-07-27T19:18:04.513411Z    credit             2.0      limit   

  opening_strategy closing_strategy  price  processed_quantity  
0              NaN        long_call   2.56                 1.0  
1        long_call              NaN   1.60    

In [10]:
options_ticker_list = []
for index, row in options_df.iterrows():
    options_ticker_list.append(row['chain_symbol'])
options_ticker_list = sorted(list(set(options_ticker_list)))
print(options_ticker_list)


['AGTC', 'AMAT', 'AMD', 'APHA', 'ASO', 'ATVI', 'BB', 'BNTX', 'CCIV', 'CCJ', 'CHPT', 'CLOV', 'CRSR', 'EBAY', 'ECVT', 'FB', 'FSLY', 'GHVI', 'GME', 'GOEV', 'HOOD', 'ICLN', 'IPOD', 'IPOF', 'ITUB', 'JMIA', 'KBH', 'LAZR', 'LCID', 'LILM', 'MARA', 'MCFE', 'MGNI', 'MT', 'MTCH', 'NIO', 'NNDM', 'NOK', 'ORCL', 'PD', 'PLTR', 'PLUG', 'PSTH', 'PTON', 'QQQ', 'QQQJ', 'RIOT', 'SKLZ', 'SNAP', 'SOLO', 'SOS', 'SPY', 'STPK', 'TDOC', 'TLRY', 'TSLA', 'UPS', 'UWMC', 'VIAC', 'VLDR', 'WISH', 'WKHS', 'WMT', 'WOOF']


In [11]:
def option_buy_or_sell(ticker, enter_or_exit):
    option = options_df.loc[(options_df['chain_symbol'] == ticker) & ((options_df[enter_or_exit] == 'long_call') | (options_df[enter_or_exit] == 'long_put'))]
    option['total_cost'] = option['processed_quantity'] * option['price'] * 100
    return option

def option_gain(ticker):
    option_buy = option_buy_or_sell(ticker, 'opening_strategy')
    sum_buy = option_buy['total_cost'].sum()
    option_sell = option_buy_or_sell(ticker, 'closing_strategy')
    sum_sell = option_sell['total_cost'].sum()
    net_value = round((sum_sell - sum_buy), 1)
    return net_value

options_gain_ticker = {}
total_gain = 0
for ticker in options_ticker_list:
    net_val = option_gain(ticker)
    options_gain_ticker.update({ticker : net_val})
    total_gain += net_val

sorted_option_tuples = sorted(options_gain_ticker.items(), key=lambda item: item[1])
sorted_option_dict = {k: v for k, v in sorted_option_tuples}

print(sorted_option_dict)
print(f'Total options realized gain: {total_gain}')

{'PSTH': -3860.0, 'NNDM': -1525.0, 'BB': -1356.0, 'LAZR': -870.0, 'BNTX': -860.0, 'ITUB': -860.0, 'RIOT': -765.0, 'JMIA': -665.0, 'SKLZ': -550.0, 'UPS': -512.0, 'TSLA': -454.0, 'WOOF': -435.0, 'IPOF': -416.0, 'ECVT': -355.0, 'FB': -355.0, 'VIAC': -342.0, 'UWMC': -325.0, 'WISH': -305.0, 'IPOD': -275.0, 'TDOC': -272.0, 'CLOV': -263.0, 'KBH': -260.0, 'AGTC': -225.0, 'WKHS': -211.0, 'MTCH': -205.0, 'FSLY': -199.0, 'QQQJ': -195.0, 'STPK': -195.0, 'CHPT': -167.0, 'EBAY': -163.0, 'WMT': -118.0, 'GME': -100.0, 'MCFE': -100.0, 'CCJ': -78.0, 'SNAP': -76.0, 'QQQ': -55.0, 'GOEV': -50.0, 'PD': -50.0, 'VLDR': -40.0, 'ORCL': -38.0, 'LILM': -20.0, 'MT': -18.0, 'AMAT': 0.0, 'GHVI': 0.0, 'NOK': 8.0, 'APHA': 21.0, 'HOOD': 38.0, 'ATVI': 41.0, 'ASO': 65.0, 'LCID': 80.0, 'TLRY': 140.0, 'MARA': 159.0, 'PLUG': 290.0, 'SOLO': 290.0, 'CRSR': 350.0, 'AMD': 505.0, 'MGNI': 575.0, 'CCIV': 750.0, 'ICLN': 1260.0, 'NIO': 1525.0, 'PLTR': 1655.0, 'SPY': 1830.0, 'PTON': 2537.0, 'SOS': 3290.0}
Total options realized gain: